In [7]:
from collections import defaultdict

import pandas as pd
from textblob import TextBlob

from iliad_network.components.analyzers import SentimentAnalyzer
from iliad_network.components.preprocessors import TextPreProcessor
from iliad_network.components.readers import LocalReader

In [2]:
!python --version

Python 3.12.8


In [3]:
reader = LocalReader()
try:
    poem_text = reader.run()
    clean_text = poem_text.content  # Access the clean text
except ValueError as e:
    print(f"Validation Error: {e}")

In [4]:
preprocessor = TextPreProcessor(model_name="en_core_web_sm")
iliad_sentences, iliad_words = preprocessor.clean_text(clean_text)
characters = preprocessor.extract_characters(clean_text)

In [ ]:
analyzer = SentimentAnalyzer(preprocessor=preprocessor)

In [ ]:
def build_cooccurrence_matrix(sentences, characters):
    # Create a dictionary to store co-occurrences
    cooccurrence_dict = defaultdict(lambda: defaultdict(int))

    # Loop over sentences
    for sentence in sentences:
        # Extract character mentions in the sentence
        mentioned_characters = set(preprocessor.extract_characters(sentence))

        # If multiple characters are mentioned, increment co-occurrence counts
        for char1 in mentioned_characters:
            for char2 in mentioned_characters:
                if char1 != char2:
                    cooccurrence_dict[char1][char2] += 1

    # Convert the dictionary into a DataFrame for easy viewing
    cooccurrence_df = pd.DataFrame.from_dict(cooccurrence_dict).fillna(0)
    return cooccurrence_df


# Example usage
cooccurrence_matrix = build_cooccurrence_matrix(iliad_sentences, characters)
print(cooccurrence_matrix.head())  # Preview the co-occurrence matrix

In [ ]:
def analyze_sentiment(sentence):
    blob = TextBlob(sentence)
    return blob.sentiment.polarity  # Returns sentiment score (-1 to 1)


def sentiment_analysis(sentences, character):
    # Dictionary to store sentiment scores for each character's sentence
    character_sentiments = []

    for sentence in sentences:
        # Check if the character is mentioned in the sentence
        if character in sentence:
            sentiment_score = analyze_sentiment(sentence)
            character_sentiments.append((sentence, sentiment_score))

    return character_sentiments


# Example usage
char = "Achilles"  # Example character
achilles_sentiments = sentiment_analysis(iliad_sentences, char)

# Print a few example sentences with sentiment scores
for sentence, sentiment in achilles_sentiments[:5]:
    print(f"Sentence: {sentence} | Sentiment: {sentiment}")